<a href="https://colab.research.google.com/github/chueneelvin/Databricks/blob/main/Multi_PDF_QnA_with_Langchain_and_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [37]:
!pip -q install langchain pypdf langchain-community langchain-text-splitters langchain_experimental langchain_openai langchain-chroma langchain-pinecone python-dotenv chromadb faiss-cpu unstructured[pdf] poppler-utils langsmith tesseract dedoc

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 9.4 MB/s eta 0:00:00
   

# Importing the dependecies

In [2]:
from langchain_community.document_loaders import PyPDFLoader        # Loading the docuements
from langchain_community.document_loaders import DirectoryLoader   # Loading the documents from a directory
from langchain_text_splitters import RecursiveCharacterTextSplitter # Text chunks using recursive spliter
from langchain_experimental.text_splitter import SemanticChunker    # Semantic text chuncking
from langchain_openai import OpenAIEmbeddings                       # openai embedding models
from langchain_chroma import Chroma                                 # vector database Chromadb
from langchain.vectorstores import Pinecone                  # vector database Pinecone
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
import pinecone
import os
from langchain.chains import StuffDocumentsChain


# Loading the data

## loading files (pdf)

In [61]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, BSHTMLLoader, CSVLoader, DedocFileLoader

# Define a dictionary to map file extensions to their respective loader classes
loader_classes = {
    "*.pdf": PyPDFLoader,
    "*.txt": TextLoader,
    "*.html": BSHTMLLoader,
    "*.csv": CSVLoader,
    "*.docx": DedocFileLoader
}

# Function to load files based on their extensions and store them in a variable
def load_files(directory):
    loaded_files = []
    for pattern, loader_cls in loader_classes.items():
        loader = DirectoryLoader(directory, glob=pattern, loader_cls=loader_cls)
        # Append the loaded files to the list
        loaded_files.extend(loader.load())
    return loaded_files

# Load files from the specified directory and store them in a variable
docs = load_files('/content/pdfs')

# Now `files` contains all the loaded files
len(docs)


1024

In [64]:
docs[1003]

Document(metadata={'source': '/content/pdfs/df_extracted (version 1).csv', 'row': 982}, page_content='tweet_id: ID_1110949565694398464,"\'nebraska\', \'nebraska\'"\nlocation: None')

# Text Chunking

## Recursive chuncking

In [65]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=15
)

chunked_docs = text_splitter.split_documents(docs)
len(chunked_docs)

1062

In [66]:
chunked_docs[:3]

[Document(metadata={'source': '/content/pdfs/Farming Potatoes in South Africa_ What You Need to Know.pdf', 'page': 0}, page_content="\uf1bf\nFarming P otatoes In South Africa: What\nYou Need To Know\n“My idea of hea ven is a gr eat big bak ed potat o and\nsomeone t o shar e it with. ”\n- Opr ah Winfr ey\nWhat's in this guide?\n1. Introduction: Farming potatoes in South Africa\n2. All about seed potatoes\n3. Growing potatoes in South Africa\n4. Challenges of potato farming\n5. Sustainable potato farming\nChapter 1\nIntroduction: F arming P otatoes in South Africa"),
 Document(metadata={'source': '/content/pdfs/Farming Potatoes in South Africa_ What You Need to Know.pdf', 'page': 0}, page_content='In South Africa, maiz e meal and br ead ar e the most commonly consumed sour ces of\ncarbohy drates. Howe ver, South Africans ha ve eaten twice as many potat oes o ver the past\ndecade compar ed to the decade befor e it, buo yed b y a gr owing middle class.'),
 Document(metadata={'source': '/co

## Semantic chunking

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter1 = SemanticChunker(OpenAIEmbeddings())
text_splitter1

In [ ]:
semantic_chunk_docs = text_splitter1.split_documents(docs)
semantic_chunk_docs[:3]

In [ ]:
len(semantic_chunk_docs)

# Convert chunks into embeddings and store into FAISS vector store

In [67]:
# Get the API key from user data
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [68]:
# Initialising the embeddings model

embeddings_model = OpenAIEmbeddings(model = "text-embedding-3-small")
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7cd8662d64d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7cd84354fa90>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Recursive Character Splitting for Chunks

In [69]:
faiss_db = FAISS.from_documents(chunked_docs, embeddings_model)
faiss_db

In [70]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")
#from langchain.prompts import PromptTemplate
#from langchain.chains import LLMChain
#llm = OpenAI()
#chain = StuffDocumentsChain(llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template("Summarize this content: {context}")), verbose=True)
#chain

In [74]:
query= "what do i need to know about potato farming as a new farmer?"
results = faiss_db.similarity_search(query)
chain.invoke({'input_documents': results, 'question': query})['output_text']

' As a new farmer, you should know that potato farming in South Africa has its own unique challenges and requires careful planning and consideration. Some important things to keep in mind include the types of potatoes you can grow, the role of potatoes in the food system, the stages of growth and development of potato plants, and the factors that affect the choice of cultivar to use. Additionally, it is important to be aware of the costs involved in potato production and to use sustainable practices to ensure a successful harvest. It may also be helpful to consult with experienced potato farmers or attend workshops or seminars on potato farming.'

## Semantic Character splitting for document chunking

# Persist the vector embeddings into a  vector database

## ChromaDB

(install via pip : pip install langchain-chroma)

## Pinecone

### Initialize the pinecone vector db

In [80]:
from datetime import date

for doc in chunked_docs:
  if doc.metadata.get("last_printed_date") is None:
    # Get today's date in YYYY-MM-DD format
    doc.metadata["last_printed_date"] = date.today().strftime("%Y-%m-%d")

In [81]:
from langchain_pinecone import PineconeVectorStore
index_name = "pdfs"
PineconeVectorStore.from_documents(chunked_docs,embeddings_model,index_name=index_name)


In [84]:
pinecone_vectorstore = PineconeVectorStore(embedding=embeddings_model,index_name=index_name)

In [85]:
pinecone_results = pinecone_vectorstore.similarity_search(query)
chain.invoke({'input_documents': pinecone_results, 'question': query})['output_text']

' As a new farmer, you need to know about the different types of potatoes that can be grown, the stages of growth and development of the potato plant, the challenges that come with potato farming, and sustainable farming practices. It is also important to choose the right cultivar based on factors such as disease resistance, climate, and intended use of the crop. Additionally, understanding the market demand for potatoes and ensuring proper storage and transportation methods are crucial for success in potato farming.'

In [ ]:
# Deleting the pinecone index
from pinecone import Pinecone

pc = Pinecone(api_key='8c3ac13f-780e-4b95-984e-90be091e60e7')
pc.delete_index('chain')

In [ ]:
pc

## Single store